In [1]:
import pickle
import random
from sklearn import metrics
from collections import defaultdict

/Users/songlin/anaconda2/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/songlin/anaconda2/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/songlin/anaconda2/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/songlin/anaconda2/envs/Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Load the ground truths data (Edge statistics)
Note the edge weights are symmetric

In [2]:
def loadfile(path):
    with open(path,'rb') as picklefile:
        file = pickle.load(picklefile, encoding='latin1')
    return file

In [3]:
q1 = loadfile('./data/2016_q1_newman-weights_kcore.pickle')
q2 = loadfile('./data/2016_q2_newman-weights_kcore.pickle')
q1_cci = loadfile('./data/2016_q1_cci-weights_kcore.pickle')

q1_cci_dict = defaultdict(float) # convert to dict for consistency
for item in q1_cci:
    q1_cci_dict[item[0]] = item[1]

## Define the accuracy functions

In [8]:
def accuracy_helper(q1, q2):
    q1_edges = set(list((u,v) for u,v in q1.keys() if u<v))
    q2_edges = set(list((u,v) for u,v in q2.keys() if u<v))
    n = len((set(q2_edges).difference(q1_edges))) # Number of new edges
    print ("Number of edges:\nQ1:", len(q1_edges), "\nQ2:", len(q2_edges), '\nNew Edges:',n)
    return q1_edges, q2_edges, n

In [5]:
#random graph:
def random_graph_accuracy(pa_score, q1_edges, q2_edges, n):
    q1_new = list(set(list(pa_score.keys())).difference(set(list(q1_edges))))
    rand_pair = random.sample(q1_new, n)
    k = list(set(rand_pair).intersection(q2_edges))
    acc_rand = len(k)*1.0 / n
    print ("Random Graph Accuracy:", acc_rand)
    return acc_rand   

In [6]:
#find the ranked list for q2_new
def check_accuracy(score, pa_score, q1_edges, q2_edges, n, reverse=True):
    #n = len((set(q2_edges).difference(q1_edges))) # Number of new edges
    q1_new = list(set(list(score.keys())).difference(set(list(q1_edges))))
    new = {(e[0],e[1]): score[(e[0],e[1])] for e in q1_new}
    sorted_pair = sorted(new.items(), key=lambda x: x[1],reverse = reverse)[:n] #top n pairs
    rank = list(set([i[0] for i in sorted_pair]).intersection(q2_edges)) 
    accuracy = len(rank)*1.0/n
    
    pairs = pa_score.keys()
    truth = [edge in q2_edges for edge in pairs]
    scores = [score[edge] for edge in pairs]
    fpr, tpr, thres = metrics.roc_curve(truth,scores)
    auc = metrics.auc(fpr,tpr)
    
    print (accuracy, auc)
    return accuracy, auc

In [25]:
len(nm_pa)

4658878

In [46]:
q1_edges, q2_edges, n = accuracy_helper(q1_cci_dict, q2)

Number of edges:
Q1: 529154 
Q2: 217525 
New Edges: 26971


In [ ]:
def check_accuracy(score, pa_score, q1_edges, q2_edges, n, reverse=True):
    pairs = pa_score.keys()
    truth = [edge in q2_edges for edge in pairs]
    scores = [score[edge] for edge in pairs]
    fpr, tpr, thres = metrics.roc_curve(truth,scores)
    auc = metrics.auc(fpr,tpr)
    return accuracy, auc

In [27]:
acc_rand = random_graph_accuracy(nm_pa, q1_edges, q2_edges, n)

Random Graph Accuracy: 0.006525527418338215


## Accuracy for newman scores

In [12]:
nm_cn = loadfile('./data/newman_cn_kcore.pickle')
nm_aa = loadfile('./data/newman_adam_kcore.pickle')
nm_jc = loadfile('./data/newman_jaccard_kcore.pickle')
nm_pa = loadfile('./data/newman_pa_kcore.pickle')

In [14]:
acc_nm_cn = check_accuracy(nm_cn, nm_pa, q1_edges, q2_edges, n)
acc_nm_aa = check_accuracy(nm_aa, nm_pa, q1_edges, q2_edges, n)
acc_nm_jc = check_accuracy(nm_jc, nm_pa, q1_edges, q2_edges, n)
acc_nm_pa = check_accuracy(nm_pa, nm_pa, q1_edges, q2_edges, n)

0.18423491898706018 0.9556389221779336
0.18842460420451596 0.9564058215178035
0.16884802194950133 0.9491101755479732
0.012754439954024694 0.6961445823526303


## Accuracy for unweighted scores

In [15]:
un_cn = loadfile('./data/sim_unweighted_cn.pickle')
un_aa = loadfile('./data/sim_unweighted_aa.pickle')
un_jc = loadfile('./data/sim_unweighted_jc.pickle')
un_pa = loadfile('./data/sim_unweighted_pa.pickle')

In [16]:
acc_un_cn = check_accuracy(un_cn, nm_pa, q1_edges, q2_edges, n)
acc_un_aa = check_accuracy(un_aa, nm_pa, q1_edges, q2_edges, n)
acc_un_jc = check_accuracy(un_jc, nm_pa, q1_edges, q2_edges, n)
acc_un_pa = check_accuracy(un_pa, nm_pa, q1_edges, q2_edges, n)

0.1800081569092729 0.9485032959137065
0.18011938749026732 0.9489987474730646
0.17615216343480034 0.949247114847912
0.1598383448889548 0.7829558269259316


## Accuracy for CCI scores

In [18]:
cci_aa = loadfile('./data/CII_adam.pickle')
cci_pa = loadfile('./data/CII_pa.pickle')
cci_cn = loadfile('./data/CII_cn.pickle')
cci_jc = loadfile('./data/CII_jaccard.pickle')

In [19]:
acc_cci_cn = check_accuracy(cci_cn, nm_pa, q1_edges, q2_edges, n)
acc_cci_aa = check_accuracy(cci_aa, nm_pa, q1_edges, q2_edges, n)
acc_cci_jc = check_accuracy(cci_jc, nm_pa, q1_edges, q2_edges, n)
acc_cci_pa = check_accuracy(cci_pa, nm_pa, q1_edges, q2_edges, n)

0.1851989173556783 0.9525806300198585
0.1854584553779986 0.9529804869784856
0.1783396981943569 0.9501873510295702
0.04445515553742909 0.7919096311862933


## Accuracy for Newman Katz

In [42]:
newman_katz_weighted_0_05 = loadfile('./data/newman_katz_weighted_0_05.pickle')
newman_katz_weighted_0_005 = loadfile('./data/newman_katz_weighted_0_005.pickle')
newman_katz_weighted_0_0005 = loadfile('./data/newman_katz_weighted_0_0005.pickle')
newman_katz_weighted_0_00005 = loadfile('./data/newman_katz_weighted_0_00005.pickle')

In [47]:
acc_newman_katz_weighted_0_05 = check_accuracy(newman_katz_weighted_0_05, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_weighted_0_005 = check_accuracy(newman_katz_weighted_0_005, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_weighted_0_0005 = check_accuracy(newman_katz_weighted_0_0005, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_weighted_0_00005 = check_accuracy(newman_katz_weighted_0_00005, nm_pa, q1_edges, q2_edges, n)

0.13306885172963553 0.4094102540503908
0.1657335656816581 0.9569792186661331
0.16892217567016424 0.9560866677657379
0.16884802194950133 0.9559671705674027


## Accuracy for the unweighted score

In [44]:
newman_katz_unweighted_0_05 = loadfile('./data/newman_katz_unweighted_0_05.pickle')
newman_katz_unweighted_0_005 = loadfile('./data/newman_katz_unweighted_0_005.pickle')
newman_katz_unweighted_0_0005 = loadfile('./data/newman_katz_unweighted_0_0005.pickle')
newman_katz_unweighted_0_00005 = loadfile('./data/newman_katz_unweighted_0_00005.pickle')

In [50]:
acc_newman_katz_unweighted_0_05 = check_accuracy(newman_katz_unweighted_0_05, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_unweighted_0_005 = check_accuracy(newman_katz_unweighted_0_005, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_unweighted_0_0005 = check_accuracy(newman_katz_unweighted_0_0005, nm_pa, q1_edges, q2_edges, n)
acc_newman_katz_weighted_0_00005 = check_accuracy(newman_katz_weighted_0_00005, nm_pa, q1_edges, q2_edges, n)

0.09232138222535316 0.38984587235314355
0.1248748655963813 0.3880018971517685
0.17926661970264357 0.9518566951156596
0.16884802194950133 0.9559671705674027


## Accuracy for the CCI Katz

In [51]:
cci_katz_weighted_0_05 = loadfile('./data/cci_katz_weighted_0_05.pickle')
cci_katz_weighted_0_005 = loadfile('./data/cci_katz_weighted_0_005.pickle')
cci_katz_weighted_0_0005 = loadfile('./data/cci_katz_weighted_0_0005.pickle')
cci_katz_weighted_0_00005 = loadfile('./data/cci_katz_weighted_0_00005.pickle')

In [52]:
acc_cci_katz_weighted_0_05 = check_accuracy(cci_katz_weighted_0_05, nm_pa, q1_edges, q2_edges, n)
acc_cci_katz_weighted_0_005 = check_accuracy(cci_katz_weighted_0_005, nm_pa, q1_edges, q2_edges, n)
acc_cci_katz_weighted_0_0005 = check_accuracy(cci_katz_weighted_0_0005, nm_pa, q1_edges, q2_edges, n)
acc_cci_katz_weighted_0_00005 = check_accuracy(cci_katz_weighted_0_00005, nm_pa, q1_edges, q2_edges, n)

0.07137295613807423 0.4606457968983157
0.15164435875570056 0.3471260948416254
0.1201290274739535 0.3180959862258936
0.18327092061844202 0.9613246701311938


## Check the node2vec

In [54]:
n2v_un_p1_q1 = loadfile('./data/node2vec-unweighted-p1-q1-100-100.pickle')

In [55]:
acc_n2v_un_p1_q1 = check_accuracy(n2v_un_p1_q1, nm_pa, q1_edges, q2_edges, n, reverse=False)

0.10381520892810797 0.0839315672463188


In [57]:
n2v_p01_q10 = loadfile('./data/node2vec-weighted-p01-q10-100-100.pickle')
n2v_p10_q10 = loadfile('./data/node2vec-weighted-p10-q10-100-100.pickle')
n2v_p10_q01 = loadfile('./data/node2vec-weighted-p10-q01-100-100.pickle')
n2v_p01_q01 = loadfile('./data/node2vec-weighted-p1-q1-100-100.pickle')

In [58]:
acc_n2v_p01_q10 = check_accuracy(n2v_p01_q10, nm_pa, q1_edges, q2_edges, n, reverse=False)
acc_n2v_p10_q10 = check_accuracy(n2v_p10_q10, nm_pa, q1_edges, q2_edges, n, reverse=False)
acc_n2v_p10_q01 = check_accuracy(n2v_p10_q01, nm_pa, q1_edges, q2_edges, n, reverse=False)
acc_n2v_p01_q01 = check_accuracy(n2v_p01_q01, nm_pa, q1_edges, q2_edges, n, reverse=False)

0.11304734715064328 0.07171991759984335
0.12761855326090987 0.06253390604451038
0.1282488598865448 0.06189715738078934
0.12765563012124134 0.06217682162114988
